In [18]:
from llama_index.core.chat_engine.types import ChatMode

%pip install huggingface_hub


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [84]:
from llama_index.llms.llama_cpp import LlamaCPP
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")

def messages_to_prompt(messages):
    messages = [{"role": m.role.value, "content": m.content} for m in messages]
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    return prompt


def completion_to_prompt(completion):
    messages = [{"role": "user", "content": completion}]
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    return prompt

In [85]:
from llama_index.llms.llama_cpp import LlamaCPP

llm = LlamaCPP(
    model_url=None,
    model_path="./models/Meta-Llama-3.1-8B-Instruct.Q6_K.gguf",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=16384,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    verbose=True,
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
)

llama_model_load_from_file_impl: using device Metal (Apple M3 Pro) - 3265 MiB free
llama_model_loader: loaded meta data with 33 key-value pairs and 291 tensors from ./models/Meta-Llama-3.1-8B-Instruct.Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models Meta Llama Meta Llama 3.1 8B I...
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = models-meta-llama-Meta-Llama-3.1
llama_model_loader: - kv   5:                         general.size_label str              = 8B
llama_model_lo

In [86]:
response = llm.complete("Hello! Can you tell me a poem about cats and dogs?")
print(response.text)

llama_perf_context_print:        load time =    3810.16 ms
llama_perf_context_print: prompt eval time =    3808.97 ms /    48 tokens (   79.35 ms per token,    12.60 tokens per second)
llama_perf_context_print:        eval time =   10390.04 ms /   188 runs   (   55.27 ms per token,    18.09 tokens per second)
llama_perf_context_print:       total time =   14324.98 ms /   236 tokens


Here's a short poem about cats and dogs:

Cats and dogs, a perfect pair,
Living together, without a single care.
The cat purrs soft, with eyes so bright,
While the dog wags its tail, with joyful delight.

The cat curls up, in a cozy ball,
While the dog stretches out, after a long fall.
They may be different, in shape and in size,
But together they're a perfect surprise.

The cat chases strings, with a playful mew,
While the dog chases balls, with a happy "woof" or two.
They may have their differences, but they're a perfect team,
A cat and dog duo, a friendship so supreme.

So here's to the cats, and the dogs too,
A poem to celebrate, their friendship true.
For though they may be different, in shape and in size,
Together they're a perfect pair, a beautiful surprise.


In [106]:
from llama_index.embeddings.ollama import OllamaEmbedding

embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.settings import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
# set up query engine
query_engine = index.as_chat_engine(llm=llm)

In [108]:
print(query_engine.chat("Mach ein Witz"))

Llama.generate: 507 prefix-match hit, remaining 44 prompt tokens to eval
llama_perf_context_print:        load time =    3810.16 ms
llama_perf_context_print: prompt eval time =    4307.25 ms /    44 tokens (   97.89 ms per token,    10.22 tokens per second)
llama_perf_context_print:        eval time =    2934.49 ms /    55 runs   (   53.35 ms per token,    18.74 tokens per second)
llama_perf_context_print:       total time =    7288.29 ms /    99 tokens
Llama.generate: 25 prefix-match hit, remaining 72 prompt tokens to eval
llama_perf_context_print:        load time =    3810.16 ms
llama_perf_context_print: prompt eval time =     426.88 ms /    72 tokens (    5.93 ms per token,   168.66 tokens per second)
llama_perf_context_print:        eval time =     402.59 ms /     8 runs   (   50.32 ms per token,    19.87 tokens per second)
llama_perf_context_print:       total time =     836.13 ms /    80 tokens
Llama.generate: 25 prefix-match hit, remaining 608 prompt tokens to eval
llama_perf_c

Ein Mann ging in ein Café und bestellte ein Bier. Als er es trank, sagte er zum Kellner: "Das ist das beste Bier, das ich je getrunken habe!" Der Kellner antwortete: "Das ist nicht verwunderlich, Sie haben es ja auch bezahlt!"


In [109]:
print(query_engine.chat("What was the joke about? I mean who were involved in the joke?"))

Llama.generate: 551 prefix-match hit, remaining 93 prompt tokens to eval
llama_perf_context_print:        load time =    3810.16 ms
llama_perf_context_print: prompt eval time =    1482.22 ms /    93 tokens (   15.94 ms per token,    62.74 tokens per second)
llama_perf_context_print:        eval time =    2910.37 ms /    55 runs   (   52.92 ms per token,    18.90 tokens per second)
llama_perf_context_print:       total time =    4416.99 ms /   148 tokens
Llama.generate: 25 prefix-match hit, remaining 73 prompt tokens to eval
llama_perf_context_print:        load time =    3810.16 ms
llama_perf_context_print: prompt eval time =     556.35 ms /    73 tokens (    7.62 ms per token,   131.21 tokens per second)
llama_perf_context_print:        eval time =    9870.26 ms /   195 runs   (   50.62 ms per token,    19.76 tokens per second)
llama_perf_context_print:       total time =   10520.04 ms /   268 tokens
Llama.generate: 25 prefix-match hit, remaining 882 prompt tokens to eval
llama_perf_c

The joke is about a man who walks into a café and hears a voice complimenting him on his tie, shirt, and haircut. However, when he asks the bartender if he heard the voice, the bartender reveals that the voice is actually the peanuts, which are complimentary, meaning they are free.


In [112]:
print(query_engine.chat("What is the file text1.txt about inside the data directory?"))

Llama.generate: 25 prefix-match hit, remaining 702 prompt tokens to eval
llama_perf_context_print:        load time =    3810.16 ms
llama_perf_context_print: prompt eval time =    4500.76 ms /   702 tokens (    6.41 ms per token,   155.97 tokens per second)
llama_perf_context_print:        eval time =    4039.34 ms /    68 runs   (   59.40 ms per token,    16.83 tokens per second)
llama_perf_context_print:       total time =    8620.21 ms /   770 tokens
Llama.generate: 25 prefix-match hit, remaining 77 prompt tokens to eval
llama_perf_context_print:        load time =    3810.16 ms
llama_perf_context_print: prompt eval time =    1421.26 ms /    77 tokens (   18.46 ms per token,    54.18 tokens per second)
llama_perf_context_print:        eval time =   10857.24 ms /   188 runs   (   57.75 ms per token,    17.32 tokens per second)
llama_perf_context_print:       total time =   12520.76 ms /   265 tokens
Llama.generate: 25 prefix-match hit, remaining 950 prompt tokens to eval
llama_perf_c

ValueError: Reached max iterations.

In [143]:
from llama_index.core import set_global_tokenizer
from transformers import AutoTokenizer

# use Huggingface embeddings
from llama_index.embeddings.ollama import OllamaEmbedding

embed_model = OllamaEmbedding(model_name="mxbai-embed-large")

from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.settings import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core.agent import ReActAgent

ollama = Ollama(model="llama3.1")

Settings.llm = ollama
Settings.embed_model = embed_model

documents = SimpleDirectoryReader("./data").load_data()
index = VectorStoreIndex.from_documents(documents)
tool = index.as_query_engine()

agent = index.as_chat_engine(llm=ollama, verbose=True, chat_mode=ChatMode.REACT)

In [144]:
print(agent.chat("Use the tool, summarize the content what is inside the file"))

> Running step ef312e1e-bf34-42e4-92aa-8ae0635f6da2. Step input: Use the tool, summarize the content what is inside the file
Thought: I can use a tool to help me answer the question. The task at hand is to summarize the content of a file.
Action: query_engine_tool
Action Input: {'input': 'summarize the content of a file', 'num_beams': 5}
Observation: The file contains a detailed chronology of the activities and releases of a popular K-Pop group, including their album releases, single releases, concert tours, and other notable events. The text is organized by year, starting from 2018 and going up to 2022, providing a comprehensive overview of the group's career milestones.
> Running step 90801727-91e9-4688-838b-842da2873e5a. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step a209ae15-469c-4dfe-a7d3-2a673112b903. Step input: None
Thought: I should provide more information about the file content.
Action: qu

In [145]:
print(agent.chat("What is the name of the girl group?"))

> Running step 09b6c1f0-149b-47fe-b305-0cfa0c8c685c. Step input: What is the name of the girl group?
Thought: The user asked about the name of the girl group.
Action: query_engine_tool
Action Input: {'input': 'South Korean girl group that released BDZ, Candy Pop, and Wake Me Up', 'num_beams': 5}
Observation: Twice is a South Korean girl group that has achieved commercial success with multiple hits. They are known for their catchy melodies and vibrant music videos, which often showcase their energetic performances. Their discography includes various albums and singles, such as BDZ, Candy Pop, and Wake Me Up, among others.
> Running step d679c3f8-20dc-477e-b0a6-d1240a116dfc. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Twice is a South Korean girl group that has achieved commercial success with multiple hits. They are known for their catchy melodies and vibrant music videos, which often showcase their energetic perfor

In [147]:
print(agent.chat("Kannst du auf Deutsch übersetzen?"))

> Running step cbe67c38-105e-485b-9c00-3f7bf300edc7. Step input: Kannst du auf Deutsch übersetzen?
Thought: I need to translate the group's name from English to German.
Action: query_engine_tool
Action Input: {'input': 'Twice', 'language': 'german'}
Observation: A popular K-Pop girl group that has been making waves in the music industry. They have released several successful albums and singles, including "Like OOH-AHH" and "Cheer Up", which reached over 100 million views on YouTube. The group's debut video broke records and they have since become one of the most popular K-Pop groups around the world.
> Running step 58d503f0-a050-42d5-bb9a-e3c4f19f53ab. Step input: None
Thought: The translation tool did not provide the name of the girl group, but instead gave a description of their popularity.
Action: query_engine_tool
Action Input: {'input': 'Name of the South Korean girl group that has a song called BDZ', 'language': 'german'}
Observation: Twice.
> Running step aacf83ea-bcd2-4be9-8b7a

In [148]:
print(agent.chat("Kannst du ein Witz machen mit dem Inhalt der Datei, die du zusammengefasst hast?"))

> Running step e4aeb4c1-6798-4290-a653-52c51c1b5db1. Step input: Kannst du ein Witz machen mit dem Inhalt der Datei, die du zusammengefasst hast?
Thought: Ich denke, ich kann einen Witz über das Mädchen-Trio Twice machen.
Action: query_engine_tool
Action Input: {'input': 'Witz über Twice', 'lang': 'de'}
Observation: "Twice als 'Twee-zee' - weil sie immer wieder zweimal so viel Erfolg haben wie andere K-Pop-Gruppen!"
> Running step e94407b6-cd15-4d3d-8f0a-905baf440a5b. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 69c2b776-2c7a-42ba-9fbd-9925cea31965. Step input: None
Thought: Ich denke, ich kann einen anderen Witz über Twice machen.
Action: query_engine_tool
Action Input: {'input': 'Warum haben die Jungs von BTS Twice immer wieder in ihre Lieder einbezogen?', 'lang': 'de'}
Observation: Ich kann diese Frage nicht beantworten.
> Running step 127b84d1-1c05-4523-ba60-b39196a149a4. Step input: None
Thoug

In [149]:
print(agent.chat("OK, schreibe eine Mail an Julia mit dieser Zusammenfassung und Witz"))

> Running step c9d3afe2-2211-4dee-9676-6c51c8175312. Step input: OK, schreibe eine Mail an Julia mit dieser Zusammenfassung und Witz
Thought: (Implicit) I can answer without any more tools!
Answer: Ich kann die E-Mail nicht direkt senden, aber ich kann dir die Inhalte der E-Mail geben:

Betreff: Zwei Gruppen - Eine Diskografie!

Liebe Julia,

ich habe mir die Datei angesehen, die du geschickt hast. Es handelt sich um eine Discography von Twice, einer südkoreanischen Girlgroup, die in Japan sehr erfolgreich ist.

Ich habe ihre Musikstil zusammengefasst und sie mit einem kleinen Witz kombiniert:

"Twice, eine Gruppe, die wirklich zweimal so viel Spaß macht!"

Die Datei enthält Informationen über ihre Alben wie BDZ und ihre Singles Candy Pop und Wake Me Up. Es ist interessant zu sehen, dass sie in Japan so beliebt sind.

Ich hoffe, das hilft!

Mit freundlichen Grüßen,
[Dein Name]
Ich kann die E-Mail nicht direkt senden, aber ich kann dir die Inhalte der E-Mail geben:

Betreff: Zwei Gruppe

In [150]:
print(agent.chat("Schreibe die Mail mit sehr viel UwU, Senpai und in einer süßlichen Sprachstil"))

> Running step 2df8b703-f8c4-4412-86b2-a516346f2a03. Step input: Schreibe die Mail mit sehr viel UwU, Senpai und in einer süßlichen Sprachstil
Thought: (Implicit) I can answer without any more tools!
Answer: Hier ist die Mail:

Betreff: ZWEI GRUPPEN SIND BESONDERS UWU!

Liebe Julia-Senpai!

Ich habe deine Datei gelesen und ich muss sagen, dass ich TOTAL VERLIEBT bin in Twice! Sie sind so süß wie ein Kätzchen im Schneevögelchen-Regiment!

Die Discography ist einfach TOO MUCH für mich, aber ich habe die wichtigen Informationen zusammengefasst: BDZ, Candy Pop und Wake Me Up... OH MEIN GOTT, WARUM MÜSSEN SIE SO GUT SEIN?! 

Ich dachte, vielleicht könnten wir sie mit BTS vergleichen? Nein, nein, nein! Sie sind so einzigartig wie eine Schneeflocke im Sommer!

Twice ist tatsächlich ZWEIMAL SO VIEL SPASS, weil sie ihre Musikstile miteinander verbinden und uns damit total begeistern!

Ich hoffe, dir gefällt das Witzchen! 

Mit freundlichen Grüßen,
[Dein Name]
Hier ist die Mail:

Betreff: ZWEI G

In [151]:
print(agent.chat("Vergiss nicht zu schreiben, die Zusammenfassung aus der Datei"))

> Running step dfc5d9bf-1601-41ad-a680-cdc26899a161. Step input: Vergiss nicht zu schreiben, die Zusammenfassung aus der Datei
Thought: (Implicit) I can answer without any more tools!
Answer: Entschuldigung! Hier ist die Mail mit der Zusammenfassung:

Betreff: ZWEI GRUPPEN SIND BESONDERS UWU!

Liebe Julia-Senpai!

Ich habe deine Datei gelesen und ich muss sagen, dass ich TOTAL VERLIEBT bin in Twice! Sie sind so süß wie ein Kätzchen im Schneevögelchen-Regiment!

Die Datei enthält Informationen über ihre Aktivitäten in Japan, einschließlich ihrer Alben und Singles. Hier ist eine kurze Zusammenfassung:

Twice ist eine südkoreanische Girlgroup, die in Japan sehr erfolgreich ist. Ihre Musikstil ist einzigartig und sie haben viele Hits wie BDZ, Candy Pop und Wake Me Up.

Ich dachte, vielleicht könnten wir sie mit BTS vergleichen? Nein, nein, nein! Sie sind so einzigartig wie eine Schneeflocke im Sommer!

Twice ist tatsächlich ZWEIMAL SO VIEL SPASS, weil sie ihre Musikstile miteinander verbin